## CS 584 Machine Learning : Homework Project: Crime Prediction
---
### Team Members:
 * Jadhav Aditya CWID
 * Sumedha Gupta CWID


In [589]:
import pandas as pd # panda's nickname is pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from  sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import binarize
import matplotlib.pyplot as plot
from sklearn import linear_model
from IPython.display import Image 
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn import tree
import sklearn.preprocessing as prep
import pydotplus 
import math
from sklearn.linear_model import RidgeCV
import sklearn.linear_model as linear_model
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import GradientBoostingClassifier


#### Load the communities-crime-clean.csv file into a dataframe object


In [590]:
communities_crime_df = pd.read_csv('communities-crime-clean.csv')

# Sanity test we have good data
communities_crime_df.head()


,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,1,Alabastercity,7,0.01,0.61,0.21,0.83,0.02,0.01,0.41,...,0.03,0.70,0.40,0.34,0.57,0.05,0.06,0.01,0.0,0.06
1,1,AlexanderCitycity,10,0.01,0.41,0.55,0.57,0.01,0.00,0.47,...,0.00,0.93,0.66,0.82,0.84,0.11,0.03,0.01,0.0,0.14
2,1,Annistoncity,3,0.03,0.34,0.86,0.30,0.04,0.01,0.41,...,0.04,0.77,0.59,0.70,0.64,0.06,0.11,0.04,0.0,1.00
3,1,Athenscity,8,0.01,0.38,0.35,0.71,0.04,0.01,0.39,...,0.03,0.78,0.56,0.67,0.71,0.09,0.05,0.00,0.0,0.23
4,1,Auburncity,1,0.04,0.37,0.32,0.70,0.21,0.02,1.00,...,0.12,0.49,0.12,0.00,0.15,0.09,0.09,0.01,0.0,0.15


### Q1. Decision Trees
---
#### a) Create a new field “highCrime” which is true if the crime rate per capita (ViolentCrimesPerPop) is greater than 0.1, and false otherwise. What are the percentage of positive and negative instances in the dataset?

In [591]:
def setHighCrime(df):
    '''Function to set value of highCrime depending on ViolentCrimesPerPop'''
    if df['ViolentCrimesPerPop'] > 0.1:
        return True
    else:
        return False
    
# Adding a new field "highCrime"
communities_crime_df['highCrime'] = communities_crime_df.apply(setHighCrime, axis=1)

# Calculating the percentage of positive and negative instances in the dataset
percentage_intances = communities_crime_df.groupby('highCrime').size() * 100 / len(communities_crime_df)
print(percentage_intances)
print("------------------")
print("Percentage Positive Instance = {}\nPercentage Negative Instance = {} ".format(percentage_intances[1],percentage_intances[0]))


highCrime
False    37.280482
True     62.719518
dtype: float64
------------------
Percentage Positive Instance = 62.719518314099346
Percentage Negative Instance = 37.280481685900654 


#### b) Use DecisionTreeClassifier to learn a decision tree to predict highCrime on the entire dataset  
##### i. What are the training accuracy, precision, and recall for this tree?

In [592]:
#Dropping non-predictive fields as well as ViolentCrimesPerPop field 
X = communities_crime_df.drop('ViolentCrimesPerPop', axis=1).drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1).drop('highCrime', axis=1)
features = list(X.columns)
y = communities_crime_df["highCrime"]

dt_clf = DecisionTreeClassifier()

dt_clf.fit(X,y)

predicted = dt_clf.predict(X[features])
#communities_crime_df['Predicted_HighCrime'] = pd.Series(p, index=communities_crime_df.index)
#dot_data = tree.export_graphviz(dt, out_file=None)
#graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png()) 
#Thus in binary classification, the count of true negatives is C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}.
#metrics.confusion_matrix(communities_crime_df['highCrime'], predicted)
recall_score = metrics.recall_score(communities_crime_df['highCrime'], predicted)
precision_score = metrics.precision_score(communities_crime_df['highCrime'], predicted)
accuracy_score = metrics.accuracy_score(communities_crime_df['highCrime'], predicted)

print("Training Accuracy = {} Precision = {} Recall = {}".format(accuracy_score,precision_score,recall_score))

Training Accuracy = 1.0 Precision = 1.0 Recall = 1.0


##### ii. What are the main features used for classification? Can you explain why they make sense (or not)?

In [593]:
# We can check the main features used for classification by checking the feature importnaces, the higher the value,
# more important the feature would be.
# We can sort the impostamce scores in descending order and take the top most.
importances=dt_clf.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")
print(indices[:5])
print(X.columns[indices[:5]])

#print(sorted(zip(X.columns, dt_clf.feature_importances_), key=lambda x: x[1], reverse=True)[:1])
#print(importantFeatures[:1])

Feature ranking:
[44  3  5 34 29]
Index(['PctKids2Par', 'racePctWhite', 'racePctHisp', 'PctEmplManu',
       'PctLess9thGrade'],
      dtype='object')


#### c. Now apply cross-validation (cross_val_score) to do 10-fold cross-validation to estimate the out-of-training accuracy of decision tree learning for this task.  
##### i. What are the 10-fold cross-validation accuracy, precision, and recall?

In [594]:
scores = cross_val_score(dt_clf, X, y, cv=10)

precision= cross_val_score(dt_clf, X, y, cv=10, scoring='precision')
recal = cross_val_score(dt_clf, X, y, cv=10, scoring='recall')
print("Results after applying 10-fold cross-validation")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
print("Recall", np.mean(recal))
print("Precision", np.mean(precision))


Results after applying 10-fold cross-validation
Accuracy: 0.72 (+/- 0.05)
Recall 0.7816
Precision 0.786547831203


##### ii. Why are they different from the results in the previous test?
TODO  
we usually use the entire dataset for building the final model, but we use cross-validation (CV) to get a better estimate of the generalization error on new unseen data.

### 2. Linear Classification
---
#### a. Use GaussianNB to learn a Naive Bayes classifier to predict highCrime.  
##### i. What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [595]:
gnb_clf = GaussianNB()

gnb_clf.fit(X, y)
predicted_highCrime = gnb_clf.predict(X[features])
#communities_crime_df['abc'] = pd.Series(p, index=communities_crime_df.index)
#communities_crime_df
#communities_crime_df.head()
#for i in communities_crime_df['abc']:
#for index, row in communities_crime_df.iterrows():    
#    print(type(row['abc']))
#confusion_matrix(communities_crime_df['highCrime_Boolean'], communities_crime_df['abc'])
#recall_score(communities_crime_df['highCrime'], communities_crime_df['abc'])
#recall_score(a['highCrime_Boolean'], a['abc'])
#communities_crime_df.head()
#accuracy_score1 = metrics.recall_score(communities_crime_df['highCrime'], predicted)
scores = cross_val_score(gnb_clf, X, y, cv=10)
precision= cross_val_score(gnb_clf, X, y, cv=10, scoring='precision')
recal = cross_val_score(gnb_clf, X, y, cv=10, scoring='recall')
print("Results after applying 10-fold cross-validation")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
print("Recall", np.mean(recal))
print("Precision", np.mean(precision))



Results after applying 10-fold cross-validation
Accuracy: 0.76 (+/- 0.05)
Recall 0.692
Precision 0.911799814828


What are the 10 most predictive features? This can be
measured by the normalized absolute difference of means for
the feature between the two classes:
|𝜇𝑇 − 𝜇𝐹|/𝜎𝑇 + 𝜎𝐹
The larger this different, the more predictive the feature. Why
do these make sense (or not)?

In [596]:
df_predictiveFeature = communities_crime_df.drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1)

dictPredFeature = {}
# TODO collecting true and false dataset
true_df = X[df_predictiveFeature['highCrime'] == 1]
false_df = X[df_predictiveFeature['highCrime'] == 0]
for column in X:
    mean_true = true_df[column].mean()
    mean_false = false_df[column].mean()
    var_true = true_df[column].var()
    var_false = false_df[column].var()
    if(column != 'highCrime'):
        predScore = abs((mean_true - mean_false))/(math.sqrt(var_true)+math.sqrt(var_false))
        dictPredFeature[column] = predScore
MostPredFeat = sorted(dictPredFeature.items(), key=lambda x: x[1])[-10:]
for i in MostPredFeat:
    print(i)


('MalePctDivorce', 0.6165342437579264)
('PctTeen2Par', 0.6426208020335665)
('pctWInvInc', 0.6607203219208594)
('PctYoungKids2Par', 0.6646705535981033)
('TotalPctDiv', 0.6742823162675443)
('FemalePctDiv', 0.6936040623575482)
('PctIlleg', 0.7089291060645266)
('racePctWhite', 0.7348840522379364)
('PctFam2Par', 0.74516152011997)
('PctKids2Par', 0.8093364216318364)


##### TODO iii. How do these results compare with your results from decision trees, above?

#### b. Use LinearSVC to learn a linear Support Vector Machine model to predict highCrime.
##### i. What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [597]:
linearsvmclf = svm.SVC(kernel='linear', C =1.0)
accuracy_svm = cross_val_score(linearsvmclf, X, y, cv=10, scoring='accuracy').mean()
precision_svm = cross_val_score(linearsvmclf, X, y, cv=10, scoring='precision').mean()
recall_svm = cross_val_score(linearsvmclf, X, y, cv=10, scoring='recall').mean()
print ('Accuracy is', accuracy_svm)
print ('Precision is', precision_svm)
print ('Recall is', recall_svm)

Accuracy is 0.804753768844
Precision is 0.855179372789
Recall is 0.8376


#### ii. What are the 10 most predictive features? This can be measured by the absolute feature weights in the model. Why do these make sense (or not)?

In [598]:
linearsvmclf.fit(X,y)

linearsvmclf_coef_array = {}
count = 0
for i in X.columns:
    linearsvmclf_coef_array[i] = abs(linearsvmclf.coef_[0][count])
    count = count+ 1

most_predictive_features = sorted(linearsvmclf_coef_array.items(), key=lambda x: x[1])[-10:]
most_predictive_features


[('agePct12t21', 0.98442987941004556),
 ('RentHighQ', 0.99163825361111957),
 ('PersPerOccupHous', 1.0275543973528991),
 ('MalePctDivorce', 1.073578841827074),
 ('population', 1.1086454264275147),
 ('PctFam2Par', 1.5426528869036265),
 ('racepctblack', 1.7203177655788284),
 ('PctKids2Par', 2.0576723456134776),
 ('racePctWhite', 2.2351581784693813),
 ('pctWInvInc', 2.3861552101921903)]

#### iii. How do these results compare with your results from decision trees, above?

#TODO

### 3. Regression
#### a. Use LinearRegression to learn a linear model directly predicting the crime rate per capita (ViolentCrimesPerPop).
##### i. Using 10-fold cross-validation, what is the estimated meansquared- error (MSE) of the model?

In [599]:
Y = communities_crime_df['ViolentCrimesPerPop']
linear_regression = LinearRegression()
#df = df.drop('highCrime', 1)
mean_sq_err= cross_val_score(linear_regression,X, Y,None,scoring='neg_mean_squared_error',cv=10)
print("Estimated meansquared- error (MSE) of the model is",np.abs(mean_sq_err.mean()))

Estimated meansquared- error (MSE) of the model is 0.0200939693044


##### ii. What is the MSE on the training set (train on all the data then test on it all)?

In [600]:
linear_regression.fit(X,y)
print("MSE on the training set is {}".format(np.mean((linear_regression.predict(X) - Y) ** 2)))

MSE on the training set is 0.21080313028166464


##### iii. What features are most predictive of a high crime rate? A low crime rate?

In [601]:
# Taking the coefficients in an array
list_coef = np.array(linearRegression.coef_)

# We can check the most predictive features by checking the coefficients, the larger the coeff , 
# more predictive the feature.
print("Most Predictive Feature of a High Crime Rate is", X.columns[np.argmax(list_coef)])
print("Most Predictive Feature of a Low Crime Rate is", X.columns[np.argmin(list_coef)])

Most Predictive Feature of a High Crime Rate is population
Most Predictive Feature of a Low Crime Rate is numbUrban


#### b.Now use Ridge regression to reduce the amount of overfitting, using RidgeCV to pick the best alpha from among (10, 1, 0.1, 0.01, and 0.001).


In [602]:
clf = RidgeCV(alphas=(10, 1, 0.1, 0.01, 0.001))
clf.fit(X, Y)
print("Best alpha", clf.alpha_)  

#Other way to find out best aplha 
#alphas = np.array([10, 1, 0.1, 0.01, 0.001])
#model = Ridge()
#grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
#grid.fit(X, Y)
#print(grid.best_estimator_.alpha)

Best alpha 1.0


##### i. What is the estimated MSE of the model under 10-fold CV?

In [603]:
ridge_clf = linear_model.Ridge(alpha=1.0)
ridge_clf.fit(X,Y)
mean_sq_err= cross_val_score(ridge_clf,X, Y,None,scoring='neg_mean_squared_error',cv=10)
print("Estimated meansquared- error (MSE) of the model is",np.abs(mean_sq_err.mean()))


Estimated meansquared- error (MSE) of the model is 0.0197950213482


##### ii. What is the MSE on the training set (train on all the data then test on it all)?

In [604]:

print("MSE on the training set is {0:0.2f}".format(np.mean((ridge_clf.predict(X) - Y) ** 2)))

#ridge_reg=linear_model.Ridge(alpha = 0.001)
#ridge_reg.fit(X,Y)

#print("Mean squared error: %.2f"
#      % np.mean((ridge_reg.predict(X) - Y) ** 2))
#print("MSE on the training set is {}".format(np.mean((ridge_reg.predict(X) - Y) ** 2)))

MSE on the training set is 0.02


### TODO iii. What is the best alpha?
iv. What does this say about the amount of overfitting in linear
regression for this problem?

#### c. Now use polynomial features to do quadratic (second-order) polynomial regression.
##### i.What is the estimated MSE of the model under 10-fold CV?

In [605]:
poly_feature = PolynomialFeatures(degree  = 2, interaction_only = False)
poly_df = poly_feature.fit_transform(X)
ridge_reg.fit(poly_df,y)
## caluclating MSE
poly_mse= cross_val_score(linearRegression, poly_df, Y,None,scoring='neg_mean_squared_error',cv=10)
print("Estimated MSE of the model under 10-fold CV is",np.abs(poly_mse.mean()))


Estimated MSE of the model under 10-fold CV is 0.129900067249


In [606]:

print("Mean squared error: %.2f" % np.mean((ridge_reg.predict(poly_df) - Y) ** 2))

Mean squared error: 0.30


### 4. Dirty Data
#### Repeat the decision tree learning question for the full (non-clean) data set and present the results.
#### a. Are the CV results better or worse? What does this say about the effect of missing values?

In [607]:
# import dirty data
df_d = pd.read_csv('communities-crime-full.csv')
df_d.head()

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [608]:
# Making data clean 

df_d_clean = df_d.replace('?', np.nan)
df_d_clean = df_d_clean.drop('community', axis=1).drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1).drop('county', axis=1)
cols_numeric = df_d_clean.columns
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
Y = imp.fit_transform(df_d_clean)
df_d_clean = pd.DataFrame(Y)
df_d_clean.columns = cols_numeric
df_d_clean['highCrime'] = df_d_clean.apply(setHighCrime, axis=1)

percentage_intances = df_d_clean.groupby('highCrime').size() * 100 / len(df_d_clean)
print(percentage_intances)
print("------------------")
print("Percentage Positive Instance = {}\nPercentage Negative Instance = {} ".format(percentage_intances[1],percentage_intances[0]))

highCrime
False    37.261785
True     62.738215
dtype: float64
------------------
Percentage Positive Instance = 62.7382146439318
Percentage Negative Instance = 37.2617853560682 


In [609]:
y = df_d_clean["highCrime"]
X = df_d_clean.drop('highCrime', axis=1).drop('ViolentCrimesPerPop', axis=1)

dt = DecisionTreeClassifier()

dt.fit(X,y)
predicted = dt.predict(X)

recall_score = metrics.recall_score(df_d_clean['highCrime'], predicted)
precision_score = metrics.precision_score(df_d_clean['highCrime'], predicted)
accuracy_score = metrics.accuracy_score(df_d_clean['highCrime'], predicted)

print("Training Accuracy = {} Precision = {} Recall = {}".format(accuracy_score,precision_score,recall_score))

Training Accuracy = 1.0 Precision = 1.0 Recall = 1.0


In [610]:
# We can check the main features used for classification by checking the feature importnaces, the higher the value,
# more important the feature would be.
# We can sort the impostamce scores in descending order and take the top most.
importances=dt.feature_importances_
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")
print(indices[:5])
print(X.columns[indices[:5]])

Feature ranking:
[44  3  5 29 34]
Index(['PctKids2Par', 'racePctWhite', 'racePctHisp', 'PctLess9thGrade',
       'PctEmplManu'],
      dtype='object')


In [611]:
scores = cross_val_score(dt, X, y, cv=10)

precision= cross_val_score(dt, X, y, cv=10, scoring='precision')
recal = cross_val_score(dt, X, y, cv=10, scoring='recall')
print("Results after applying 10-fold cross-validation")
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
print("Recall", np.mean(recal))
print("Precision", np.mean(precision))

Results after applying 10-fold cross-validation
Accuracy: 0.75 (+/- 0.03)
Recall 0.789733333333
Precision 0.815353041254


## 5. Team Work

In [612]:
svmclf = svm.SVC(kernel='poly')
# non clean dataset 
acc_scores = cross_val_score(svmclf, X, y, cv=10, scoring='accuracy').mean()
pre_scores = cross_val_score(svmclf, X, y, cv=10, scoring='precision').mean()
rec_scores = cross_val_score(svmclf, X, y, cv=10, scoring='recall').mean()

print ('Accuracy is', acc_scores)
print ('Precision is', pre_scores)
print ('Recall is', rec_scores)

# clean dataset
X = communities_crime_df.drop('ViolentCrimesPerPop', axis=1).drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1).drop('highCrime', axis=1)
features = list(X.columns)
y = communities_crime_df["highCrime"]

acc_scores = cross_val_score(svmclf, X, y, cv=10, scoring='accuracy').mean()
pre_scores = cross_val_score(svmclf, X, y, cv=10, scoring='precision').mean()
rec_scores = cross_val_score(svmclf, X, y, cv=10, scoring='recall').mean()

print ('Accuracy is', acc_scores)
print ('Precision is', pre_scores)
print ('Recall is', rec_scores)


Accuracy is 0.65798319958
Precision is 0.647703862247
Recall is 0.9976
Accuracy is 0.691396984925
Precision is 0.674797386791
Recall is 0.9856


## Extra Credit

#### GradientBoostingClassifier

In [613]:
df = communities_crime_df.drop('ViolentCrimesPerPop', axis=1).drop('state', axis=1).drop('communityname', axis=1).drop('fold', axis=1).drop('highCrime', axis=1)
y = communities_crime_df["highCrime"]
X = df
clf = GradientBoostingClassifier()
clf.fit(X, y)
predicted = clf.predict(X)

recall_score = metrics.recall_score(communities_crime_df['highCrime'], predicted)
precision_score = metrics.precision_score(communities_crime_df['highCrime'], predicted)
accuracy_score = metrics.accuracy_score(communities_crime_df['highCrime'], predicted)

print("Training Accuracy = {} Precision = {} Recall = {}".format(accuracy_score,precision_score,recall_score))



Training Accuracy = 0.9257400903161064 Precision = 0.9408 Recall = 0.9408
